In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("kafka_to_clickhouse") 
    .master("spark://spark-master:7077")
    .config("spark.jars.packages", 
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1,"
            "org.apache.kafka:kafka-clients:3.5.1,"
            "org.apache.commons:commons-pool2:2.11.1,"
            "com.clickhouse:clickhouse-jdbc:0.7.2" # с версиями 0.8.x и более не работает
           )
    .config("spark.sql.streaming.checkpointLocation", "/tmp/checkpoint")
    # Чтобы сессия не занимала все воркеры
    .config("spark.cores.max", "3")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.commons#commons-pool2 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-920f6be9-18fe-4560-8858-87bed8a74f0c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in centr

## Подготовка таблицы в ClickHouse

Выполним DDL-запрос по адресу http://localhost:8123/play
```sql
CREATE TABLE IF NOT EXISTS default.streaming_test_table
(
    id String,
    name String,
    timestamp String,
    kafka_timestamp DateTime64(3),
    processed_at DateTime64(3)
)
ENGINE = MergeTree()
ORDER BY (processed_at, id)
PARTITION BY toYYYYMM(processed_at);
```


## Начнем запись в Kafka

Создадим новый ```test_topic``` в Kafka (подключен kafka-ui по адресу http://localhost:8090)

Выполним ```python3 main.py``` в терминале

## Запускаем стриминг (микробатчинг)

In [5]:
from pyspark.sql.functions import from_json, col, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("timestamp", StringType(), True)
])

kafka_df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "test_topic")
    .option("startingOffsets", "earliest")
    .load()
)

parsed_df = kafka_df.select(
    from_json(col("value").cast("string"), schema).alias("data"),
    col("timestamp").alias("kafka_timestamp")
).select("data.*", "kafka_timestamp")

processed_df = parsed_df.withColumn("processed_at", current_timestamp())

In [ ]:
def write_to_clickhouse(batch_df, batch_id):
    if not batch_df.isEmpty():
        print(f"\nProcessing batch {batch_id} with {batch_df.count()} records")
        (
            batch_df.write.format("jdbc")
            .option("driver", "com.clickhouse.jdbc.ClickHouseDriver")
            .option("url", "jdbc:clickhouse://clickhouse-server:8123/default")
            .option("dbtable", "default.streaming_test_table")
            .option("user", "default")
            .option("password", "1234qwe")
            .option("batchsize", "10000")
            .option("socket_timeout", "300000")
            .option("numPartitions", "4")
            .option("rewriteBatchedStatements", "true")
            .mode("append")
            .save()
        )
        print(f"Batch {batch_id} written successfully to ClickHouse")
    else:
        print(f"Batch {batch_id} is empty, skipping")

query = processed_df.writeStream \
    .foreachBatch(write_to_clickhouse) \
    .outputMode("append") \
    .option("checkpointLocation", "/tmp/checkpoint/kafka_to_clickhouse") \
    .start()

query.awaitTermination()

25/10/22 00:18:43 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Batch 0 is empty, skipping



Processing batch 1 with 1 records


Batch 1 written successfully to ClickHouse



Processing batch 2 with 5 records


...

Batch 90 written successfully to ClickHouse

Processing batch 91 with 1 records


Batch 91 written successfully to ClickHouse

Processing batch 92 with 1 records


Batch 92 written successfully to ClickHouse

Processing batch 93 with 1 records


Batch 93 written successfully to ClickHouse



Processing batch 94 with 1 records
Batch 94 written successfully to ClickHouse
